# An Agent for Automated Planning and Exercise Program Building

In [40]:
# !pip install -U langchain
# !pip install openai-whisper
# !pip install pydub
# !pip install moviepy
# !pip install pytube
# !pip install unstructured

  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached emoji-2.9.0-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached jsonpath_python-1.0.6-py3-none-any.whl (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 98.8 kB/s eta 0:00:00m eta 0:00:010:00:01m
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
Using cached emoji-2.9.0-py2.py3-none-any.whl (397 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 49.9 kB/s eta 0:00:00m eta 0:00:010:00:06m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 48.0 kB/s e

In [1]:
!mkdir resources

In [3]:
!ls -d */

resources/


Let's organize the resources.

Resources involve:

- Personal notes on objectives and goals (maybe this should be in the systems prompt?)
- PDFs for books and papers related to health, muscle building, strength training, wrestling and grappling resources
- Table with information about my progress
- Transcripts of Youtube videos (built programatically from the Youtube URLs)
- Exercise descriptions (preferably with reference resources)
- Perhaps medically related information?
- Time available? (dynamic variable)
- Energy available (dynamic variable)

In [13]:
%%writefile youtube_urls.txt
https://www.youtube.com/watch?v=TrLV03TYocs
https://www.youtube.com/watch?
https://www.youtube.com/watch?v=0dB7JjpBMtI
https://youtu.be/pyWiE7oDJQ8?si=aq0TriysrRB8XUDl
https://www.youtube.com/shorts/pzS0L9CllwM
https://www.youtube.com/shorts/UX0k3sfiJHg
https://www.youtube.com/shorts/4kfVHtwm_TY
https://www.youtube.com/watch?v=fW79HCBIY-c
https://www.youtube.com/watch?v=0dB7JjpBMtI
https://www.youtube.com/watch?v=8y2HSu6llZE
https://www.youtube.com/watch?v=tf__0rFBlpc
https://www.youtube.com/watch?v=pyGvRGATPHs
https://www.youtube.com/watch?v=YeIu6CX5AOo
https://www.youtube.com/watch?v=fUOEdGmgy98˚

Overwriting youtube_urls.txt


Let's process all these youtube videos in the following manner:

1. Download
2. Transcribe
3. Save Transcription to .txt file
4. Delete downloaded video

In [7]:
with open('youtube_urls.txt') as f:
    urls = f.readlines()

In [8]:
urls

['https://www.youtube.com/watch?v=TrLV03TYocs\n',
 'https://www.youtube.com/watch?v=jb3lme_KIjo\n',
 'https://www.youtube.com/watch?\n',
 'https://www.youtube.com/watch?v=0dB7JjpBMtI\n',
 'https://youtu.be/pyWiE7oDJQ8?si=aq0TriysrRB8XUDl\n',
 'https://www.youtube.com/shorts/pzS0L9CllwM\n',
 'https://www.youtube.com/shorts/UX0k3sfiJHg\n',
 'https://www.youtube.com/shorts/4kfVHtwm_TY\n',
 'https://www.youtube.com/watch?v=fW79HCBIY-c\n',
 'https://www.youtube.com/watch?v=0dB7JjpBMtI\n',
 'https://www.youtube.com/watch?v=8y2HSu6llZE\n',
 'https://www.youtube.com/watch?v=tf__0rFBlpc\n']

To process these youtube videos we will use a set of tools from the youtube_utils.py script in the current folder.

In [22]:
from youtube_utils import *

In [24]:
urls = ['https://www.youtube.com/watch?v=TrLV03TYocs',
 'https://www.youtube.com/watch?v=jb3lme_KIjo',
 'https://www.youtube.com/watch?']

In [25]:
for i, url in enumerate(urls):
    try:
        transcription = transcribe_youtube_video(url)
        with open(f"youtube_vid_{i}.txt", "w") as f:
            f.write(transcription)
    except Exception as e:
        print(e)
        print(f"Failed to download/transcribe this video: {url}")

MoviePy - Writing audio in youtube_audio.mp3


MoviePy - Done.
MoviePy - Writing audio in youtube_audio.mp3


MoviePy - Done.
regex_search: could not find match for (?:v=|\/)([0-9A-Za-z_-]{11}).*
Failed to download/transcribe this video: https://www.youtube.com/watch?


In [27]:
# we check in with the downloaded transcriptions
!ls youtube_vid_*.txt

youtube_vid_0.txt youtube_vid_1.txt


In [29]:
# we delete the downloaded videos to avoid storage issues

!rm ./*.mp4

In [30]:
%%writefile web_articles.txt

https://www.healthline.com/health/uneven-shoulders?utm_source=pocket_mylist&c=444733912517#exercises
https://journals.lww.com/nsca-scj/fulltext/2011/12000/strength_and_conditioning_for_grappling_sports.4.aspx

Writing web_articles.txt


For web article we will scrape the contents of the web article page and save it locally to .txt files.

In [31]:
import requests
from bs4 import BeautifulSoup

def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

with open("web_articles.txt", "r") as f:
    web_article_urls = f.readlines()

for i,url in enumerate(web_article_urls):
    try:
        text = scrape_text(url)
        with open(f"web_article_{i}.txt", "w") as f:
            f.write(text)
    except Exception as e:
        print(e)
        print(f"Failed to download/transcribe this web article: {url}")

Invalid URL '': No scheme supplied. Perhaps you meant https://?


In [32]:
!ls web_article_*.txt

web_article_0.txt web_article_1.txt web_article_2.txt


In [33]:
%%writefile personal_notes.txt

- Create a warm up just based on a combination of shuffle step dancing and footwork like dominick cruz and Vasily Lomachenko
- Create a wrestling movement practice! Granby roll, uchimata, double leg shot, sweep single shot, getting out of bottom, footwork, sprawl, inverting.
- Create drills/games/challenges to become an expert headlock type grips.
- More hip mobility and strength
- Work on slide by muscles: '''Quadriceps: Essential for explosiveness and leg drive.
Hamstrings: Provide stability and power during the slide.
Calves: Assist with balance and quick footwork.
Abdominals: Core strength is crucial for balance and twisting.
Deltoids: Important for controlling the opponent's upper body.
Biceps and Triceps: Aid in controlling the opponent's arms.
Pectoralis Major: Helps with upper body control and stability.
Latissimus Dorsi: Provides strength in pulling and twisting.
Trapezius: Supports the shoulders and upper body control.
Rhomboids: Assist with shoulder blade stability.
Erector Spinae: Support the lower back during the move.
Gluteus Maximus: Contributes to hip movement.
Adductors: Assist with leg control and balance.
Hip Flexors: Important for hip movement and balance.
Serratus Anterior: Aids in stabilizing the shoulder girdle.
Forearm muscles: Assist with grip and control.
Neck muscles: Support head movement and stability.''' 
- Wrestling: '''
    - mat returns
    - mat return fail to arm in/high wrist guilhotine variation
    - front headlock throw
    - front headlock throw to high elbow/high wrist guilhotine while controlling the arm
- Guard
    - k guard to back side 50/50 (and finish!)
    - fake straight footlock to cross ashi
    - reverse delariva to cool cross ashi entry like in here
- defense
    - buggy chokes from everywhere (setups, like in the body lock pass) 
- reverse grip when loose passing 
- general
    - chaining submissions from top and bottom! Wrestling to submission, guard to submission!
- false reap to the back when opponent turns his back'''
- Jiu Jitsu Warmup: '''
Running/jumping jacks/rowing (5m)

- rubber band warmups (quality reps for each) (max 5m)
   - snap down
   - underhook 
   - slideby
   - uchikomi 

Hand fighting setups (5m)
    - undertook to the back (3x quality reps)
    - slide by (3x quality reps)
    - snap down (3x quality reps)
    - 2-1 (3x quality reps)
    - inside ties (3x quality reps)

- Hand fighting (3x1m)

Entries/Lifts/Throws
    - Uchimata shadow (2x1m)
    - Double leg entries (2x1m)
    - Single leg entries (2x1m)
    - Lifting from the back (2x1m)

- Turtle escapes (3x all quality reps) - 3-5m max
    - wrist in the pocket
    - switch
    - granby roll '''
- Medicine ball exercises to build strength for chokes etc...
- Practice the arch of the spine
- Kettlebell + towel exercises for grip and obliques
- Try that squat with the weight on your forearms
- Learn the macaquinho movement properly

Writing personal_notes.txt


In [35]:
!rm web_articles.txt youtube_urls.txt

Let's move all the .txt files to a `resources` folder so tha we can load everything directly from that folder.

In [36]:
!mkdir resources

mkdir: resources: File exists


In [37]:
!mv *.txt ./resources

In [38]:
!ls resources

personal_notes.txt web_article_1.txt  youtube_vid_0.txt  youtube_vid_1.txt


In [41]:
# source: https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents

# !pip install -U langchain langchain-community langchainhub openai faiss-cpu

# !pip install langchainhub

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("./resources/")
documents = loader.load_and_split()

In [42]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# text_splitter = CharacterTextSplitter(chunk_size=1000)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embeddings)

In [43]:
retriever = db.as_retriever()

Now we need to create a tool for our retriever. The main things we need to pass in are a name for the retriever as well as a description. These will both be used by the language model, so they should be informative.

In [44]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_exercise_docs",
    "Searches and retuens excerpts from notes, youtube transcripts and web articles about exercising workouts, strength training, grappling training as well as my personal objectives and goals."
)

tools = [tool]

# Agent Constructor
Here, we will use the high level create_openai_tools_agent API to construct the agent.

Notice that beside the list of tools, the only thing we need to pass in is a language model to use. Under the hood, this agent is using the OpenAI tool-calling capabilities, so we need to use a ChatOpenAI model.

In [45]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [56]:
from langchain.chat_models import ChatOpenAI

llm_chat = ChatOpenAI(model="gpt-4-preview-1106",temperature=0)

In [57]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [58]:
agent = create_openai_tools_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [59]:
result = agent_executor.invoke({"input": "hi, I am Lucas"})

In [60]:
result["output"]

'Hello Lucas! How can I assist you today?'

In [61]:
result = agent_executor.invoke({
    "input": "What are my personal goals regarding exercise and fighting?"
})

In [62]:
result["output"]

'Here are your personal goals regarding exercise and fighting:\n\n1. Create a warm-up based on a combination of shuffle step dancing and footwork like Dominick Cruz and Vasily Lomachenko.\n2. Develop a wrestling movement practice, including Granby roll, uchimata, double leg shot, sweep single shot, getting out of bottom, footwork, sprawl, inverting.\n3. Create drills/games/challenges to become an expert in headlock type grips.\n4. Improve hip mobility and strength.\n5. Work on slide by muscles, including quadriceps, hamstrings, calves, abdominals, deltoids, biceps and triceps, pectoralis major, latissimus dorsi, trapezius, rhomboids, erector spinae, gluteus maximus, adductors, hip flexors, serratus anterior, forearm muscles, and neck muscles.\n6. Practice wrestling techniques such as mat returns, front headlock throw, and guard techniques.\n7. Develop a Jiu Jitsu warmup routine, including running/jumping jacks/rowing, rubber band warmups, hand fighting setups, entries/lifts/throws, and

In [63]:
from IPython.display import Markdown

result = agent_executor.invoke({
    "input": "If you were to create a workout based on all the information present in my notes regarding \
        my interests and interesting exercise routines and workouts.\
        Considering I only have 45 minutes a day, what work out would you recommend?"
})

Markdown(result["output"])

Based on the information from your notes and other resources, here's a 45-minute workout routine that incorporates your interests and exercise routines:

**Warm-up (5 minutes)**

- Start with a combination of shuffle step dancing and footwork like Dominick Cruz and Vasily Lomachenko.

**Strength Training (20 minutes)**

- **Shoulder Bridge (3 sets of 15-30 seconds)**: This exercise strengthens the posterior chain and increases thoracic rotation. It's also known as the Jiu-Jitsu bridge or Upa in martial arts.

- **Tabletop Bridge (3 sets of 15-30 seconds)**: This exercise strengthens the abdominals and glutes while improving shoulder mobility.

- **Back Bridge (3 sets of 15-30 seconds)**: This iconic exercise is used in martial arts, gymnastics, calisthenics, and yoga.

- **Dumbbell Exercises (3 sets of 12 repetitions)**: Reverse fly and overhead external shoulder rotation exercises can help build strength in your shoulders and upper body.

**Grappling Training (15 minutes)**

- Practice wrestling movements like Granby roll, uchimata, double leg shot, sweep single shot, getting out of bottom, footwork, sprawl, and inverting.

- Work on slide by muscles and headlock type grips.

- Practice Jiu Jitsu movements like snap down, underhook, slideby, uchikomi, and hand fighting setups.

**Cool Down (5 minutes)**

- Finish with a 5-minute backward walk focusing on the big toe with each step. This can help strengthen your feet and leg muscles.

Remember, consistency is key in seeing progress in your workouts. Make sure to adjust the routine as needed to fit your current fitness level and goals.

Possible extensions of this agent could be to integrate papers, web articles, youtube videos, transcripts and more. We could also do some fancy prompt engineering to get the best possible performance from the agent. 